# La Qualità dell'Aria

Dalla fonte: https://aqicn.org/data-platform/register/ scarichiamo il dataset dei dati storici della stazione di monitoraggio di Cipro (Roma):

Nei dati sopra riportati pubblichiamo l'indice di Qualità dell'Aria "Inquinante Individuale". Ad esempio, se controlli le cifre per PM2.5, corrispondono al singolo AQI PM2.5 (indicato come AQIPM2.5). Se vuoi calcolare l'AQI complessivo, noto anche come AQI composito, devi prendere il massimo di tutto l'AQI individuale:

AQI = max (AQIPM2.5, AQIPM10, AQIO3, ...)

Grafici con https://chart-studio.plotly.com/

In [120]:
import os
import pandas as pd
import numpy as np
import chart_studio
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import IFrame

# Key and Secret
config = {} 
config_path = os.path.join(os.path.abspath('/opt/app/data/config')) 
config_name = r'config_od.py' 
config_file = os.path.join(config_path,config_name) 
exec(open(config_file).read(),config)

# Plotly settings
plotly_user=config['PLOTLY_USER']
plotly_key=config['PLOTLY_KEY']
chart_studio.tools.set_credentials_file(username=plotly_user, api_key=plotly_key)

In [121]:
#Leggo dati della stazione di monitoraggio storici
df_aqi_hist = pd.read_csv('cipro-air-quality.csv',index_col='date', sep="\s*,\s*", engine='python') #Leggo dato storico
df_aqi_hist['AQI'] = df_aqi_hist.max(axis=1) # Calcolo l'AQI
df_aqi_hist = df_aqi_hist[['AQI']] # Tengo solo AQI
df_aqi_hist.index = pd.to_datetime(df_aqi_hist.index) # Conveto indice in datetime

In [122]:
df_aqi_hist['AQI7ma'] = df_aqi_hist['AQI'].rolling(window=7, min_periods=0).mean() #Media mobile 7 giorni

In [123]:
df_aqi_hist.reset_index(inplace=True)

In [124]:
df_aqi_hist['year'] = df_aqi_hist['date'].dt.year.astype('str') # Colonna Anno
df_aqi_hist['period'] = df_aqi_hist['date'].dt.strftime('%b-%d') # Colonna Mese-Giorno

In [125]:
df_aqi_hist.sort_values(by=['date'],inplace=True)

In [126]:
# Lavoro solo su 2019 e 2020
df_aqi_2020 = df_aqi_hist[df_aqi_hist['year']=='2020'] # Anno 2020
df_aqi_2019 = df_aqi_hist[df_aqi_hist['year']=='2019'] # Anno 2019
df_aqi_2020.set_index('period', inplace=True)
df_aqi_2019.set_index('period', inplace=True)

In [127]:
df_aqi = df_aqi_2019[['AQI','AQI7ma']].join(df_aqi_2020[['AQI','AQI7ma']], how='left',lsuffix='_2019', rsuffix='_2020')

In [128]:
df_aqi.reset_index(inplace=True)

In [129]:
df_aqi.head(2)

,period,AQI_2019,AQI7ma_2019,AQI_2020,AQI7ma_2020
0,Jan-01,91.0,54.571429,112.0,53.428571
1,Jan-02,61.0,57.857143,109.0,57.857143


In [143]:
'''Trend AQI media mobile a 7 giorni'''
def trend_aqi_7ma(df_aqi):
    # Create a trace
    trace1 = go.Scatter(
        x = df_aqi['period'],
        y = df_aqi['AQI7ma_2019'],
        mode = 'lines',
        name = 'AQI-2019'
    )
    
    trace2 = go.Scatter(
        x = df_aqi['period'],
        y = df_aqi['AQI7ma_2020'],
        mode = 'lines',
        name = 'AQI-2020'
    )
    data = [trace1,trace2]
    
    # Edit the layout
    layout = dict(title = 'Media Mobile 7 giorni Air Quality Index (Cipro, Roma)',
                  yaxis = dict(title = 'Air Quality Index')
                  )
    
    annotations = []
    annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.15,
                                  xanchor='center', yanchor='top',
                                  text='Fonte: World Air Quality Index',
                                  font=dict(family='Arial',
                                            size=12,
                                            color='rgb(150,150,150)'),
                                  showarrow=False)
                      )
    layout['annotations'] = annotations
        
    fig = dict(data=data, layout=layout)

    return fig

In [144]:
fig_aiq_7ma = trend_aqi_7ma(df_aqi)
py.iplot(fig_aiq_7ma, filename='aiq-trend_7ma')